In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Load the pre-trained model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaForSequenceClassification.from_pretrained('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/myModel_better')

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [11]:
#detect emotion
def predict_emotion(text: str) -> int:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_encoding = tokenizer.encode_plus(text,
                                            truncation=True,
                                            padding=True,
                                            return_tensors='pt').to(device)
    model.to(device)
    outputs = model(input_ids=input_encoding['input_ids'],
                    attention_mask=input_encoding['attention_mask']).logits.to('cpu')
    predicted_label = torch.argmax(outputs, axis=1).item()

    # Map the predicted label to the actual emotion
    emotion_map = {0:'surprise',1:'happy', 2:'sad', 3:'anger', 4:'fear'}
    predicted_emotion = emotion_map[predicted_label]

    return predicted_emotion


In [70]:
# Load your test data
test_data = pd.read_csv('/content/gdrive/MyDrive/EXAM EMOTIONS MODEL/test.csv')

In [71]:
test_data

,text,emotion
0,I'm feeling anxious about having to retake the...,fear
1,I'm furious that the exam had questions that w...,anger
2,I'm so proud of myself for completing my final...,happy
3,"""I'm feeling really down and discouraged after...",sad
4,I'm feeling happy that my hard work and dedica...,happy
...,...,...
379,I'm furious that the exam questions were so un...,anger
380,I'm surprised that I was able to answer all th...,surprise
381,I'm delighted that I got a perfect score on my...,happy
382,I'm happy that I was able to finish my exam ea...,happy


In [72]:
test_data['emotion'].unique()

array(['fear', 'anger', 'happy', 'sad', 'surprise'], dtype=object)

In [73]:
# Make predictions on the test set
predicted_emotions = [predict_emotion(text) for text in test_data['text']]

# Calculate metrics
true_emotions = test_data['emotion'].tolist()

In [74]:
accuracy = accuracy_score(true_emotions, predicted_emotions)
conf_matrix = confusion_matrix(true_emotions, predicted_emotions)
classification_rep = classification_report(true_emotions, predicted_emotions)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.9765625
Confusion Matrix:
[[80  0  0  6  0]
 [ 0 88  0  2  0]
 [ 0  0 88  0  0]
 [ 0  1  0 78  0]
 [ 0  0  0  0 41]]
Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.93      0.96        86
        fear       0.99      0.98      0.98        90
       happy       1.00      1.00      1.00        88
         sad       0.91      0.99      0.95        79
    surprise       1.00      1.00      1.00        41

    accuracy                           0.98       384
   macro avg       0.98      0.98      0.98       384
weighted avg       0.98      0.98      0.98       384

